# Target Dataframes: 
    
    

Concise: 
Unique ID | Listing Price | Asset creation date | Sale start | Sale duration | Primary Sale (boolean) | Number of Sales | Buy or Sale | Sale type | Creator Fee

Raw:
Everything

Responses:
Get responses

This notebook
1. Read orderbook_samples.csv, set listed_before variable to earliest timestamp. If no timestamp found, 
set listed_before to current time.
2. Call Orderbook API X number of times, get Y orders before listed_before.
3. Produce target dataframes. 
4. Append new data to orderbook_samples.csv

In [6]:
import pandas as pd
import requests
import os
from datetime import datetime



In [42]:
def get_timestamp(file_name, time_step=86400):
    #get unix timestamp for the time of the earliest transaction in csv minus time_step.
    # time_step is one day by default.
    # If file doesn't exit, gets current timestamp.

    t0 = pd.Timestamp("1970-01-01")
    if os.path.exists(file_name):
        csv = pd.read_csv(file_name, index_col=False)
        data_time = pd.to_datetime(csv.tail(1)['created_date'])
        timestamps = data_time.apply(datetime.timestamp)

        return timestamps.values[-1] - time_step
    else:
        return datetime.timestamp(datetime.now())

In [44]:
def get_orders(listed_before, query_limit = 20, order_limit=50):
    #calls OpenSea orderbook API, creates a dataframe of length order_limit * query_limit
    df = pd.DataFrame()
    df_simple = pd.DataFrame()
    q = 0
    QUERY_COUNT = 0
    while True:
        url = f"https://api.opensea.io/wyvern/v1/orders?bundled=false&include_bundled=false&include_invalid=false&listed_before={listed_before}&limit={order_limit}&offset={order_limit*q}&order_by=created_date&order_direction=desc"
        headers = {"Accept": "application/json"}
        response = requests.request("GET", url, headers=headers)
        if response.ok is False:
            print('API call failed')
            return df, df_simple
        df = df.append(response_to_df(response),
                       ignore_index=True)
        df.sort_values('id', ascending = False,inplace=True)
        df_simple = drop_columns(df.copy())
        QUERY_COUNT += 1
        q += 1
        if QUERY_COUNT >= query_limit:
            # print("Query completed!")
            return df, df_simple

In [33]:
# LIMIT = 50
# listed_before = 1635621155
# url = f"https://api.opensea.io/wyvern/v1/orders?bundled=false&include_bundled=false&include_invalid=false&listed_before={listed_before}&limit={LIMIT}&offset=0&order_by=created_date&order_direction=desc"
# headers = {"Accept": "application/json"}
# response = requests.request("GET", url, headers=headers)
# df = response_to_df(response)

In [34]:
def response_to_df(response):
    # Clean up the response.json() a bit, and extracts some information from dictionary columns
    # Runs into error if value is none in column
    df = pd.DataFrame(response.json()['orders'])
    #drops rows that contain bundles
    df['bundle'] = df['metadata'].apply(lambda x: 1
                                      if 'bundle' in x.keys() else 0)
    df = df[df['bundle'] == 0]
    #creates unique id for each asset
    asset_unique_id = pd.DataFrame(pd.DataFrame(
        df["metadata"].tolist())['asset'].tolist())
    df['asset_unique_id'] = asset_unique_id[['id', 'address']].agg('-'.join,
                                                               axis=1)
    #extracts information from asset dictionary
    df['number_sales'] = pd.DataFrame(df["asset"].tolist())['num_sales']
    df['creator_fee'] = pd.DataFrame(
        pd.DataFrame(df["asset"].tolist())
        ['asset_contract'].tolist())['seller_fee_basis_points']
    df['description'] = pd.DataFrame(df["asset"].tolist())['description']
    df['external_link'] = pd.DataFrame(df["asset"].tolist())['external_link']
    df['image_url'] = pd.DataFrame(df["asset"].tolist())['image_url']
    df['animation_url'] = pd.DataFrame(df["asset"].tolist())['animation_url']
    df['sale_duration'] = pd.to_datetime(df['closing_date']) - pd.to_datetime(df['created_date'])
    #drops asset column
    df.drop(columns="asset", inplace=True)
    return df

In [35]:
def drop_columns(
    df,
    delete_columns=[
        'expiration_time', 'listing_time', 'current_bounty', 'bounty_multiple',
        'maker_relayer_fee', 'taker_relayer_fee', 'maker_protocol_fee',
        'taker_protocol_fee', 'maker_referrer_fee', 'fee_recipient',
        'fee_method', 'target', 'how_to_call', 'calldata',
        'replacement_pattern', 'static_target', 'static_extradata', 'extra',
        'quantity', 'salt', 'v', 'r', 's', 'approved_on_chain', 'cancelled',
        'finalized', 'asset_bundle', 'closing_extendable', 'order_hash',
        'metadata', 'exchange', 'maker', 'taker', 'marked_invalid',
        'prefixed_hash', 'bundle', 'asset_unique_id', 'number_sales'
    ]):
    #leave only essential columns
    for i in delete_columns:
        df.drop(columns=i, inplace=True)
    return df

In [36]:
def save_df(df, file_name, append=True):
    # Save df to a .csv, supports appending
    if append and os.path.exists(file_name):
        df_read = pd.read_csv(file_name, index_col=False)
        df_read = df_read.append(df, ignore_index=True)
        #df_read.drop_duplicates(keep="first", inplace=True) # Doesn't work for some reason! also breaks with dict
        df_read.to_csv(file_name, index=False)
    else:
        df.to_csv(file_name, index=False)


In [37]:
import progressbar

def main(iterations,
         df_file_name='orderbook_samples.csv',
         simple_df_file_name='orderbook_samples_essential.csv'):
    # run this to add rows to csv
    for i in progressbar.progressbar(range(iterations)):
        listed_before_timestamp = get_timestamp(df_file_name)
        df,df_simple = get_orders(listed_before_timestamp)
        save_df(df, df_file_name)
        save_df(df_simple, simple_df_file_name)


Note: It takes 20-40 seconds to add 1000 rows

In [46]:
main(2)

100% (2 of 2) |##########################| Elapsed Time: 0:01:36 Time:  0:01:36


In [47]:
# check csv length is correctly updated. The number of rows added may be
#   slightly less than what you expect because bundle orders are dropped.
print(pd.read_csv('orderbook_samples.csv').shape)
print(pd.read_csv('orderbook_samples_essential.csv').shape)

(3998, 53)
(3998, 27)


In [48]:
pd.read_csv('orderbook_samples_essential.csv').columns

Index(['id', 'asset_bundle', 'created_date', 'closing_date',
       'closing_extendable', 'order_hash', 'metadata', 'exchange', 'maker',
       'taker', 'current_price', 'side', 'sale_kind', 'payment_token',
       'payment_token_contract', 'base_price', 'marked_invalid',
       'prefixed_hash', 'bundle', 'asset_unique_id', 'number_sales',
       'creator_fee', 'description', 'external_link', 'image_url',
       'animation_url', 'sale_duration'],
      dtype='object')

In [ ]:
[
   'asset_bundle',  'closing_extendable',
    'order_hash', 'metadata', 'exchange', 'maker', 'taker', 'marked_invalid', 'prefixed_hash', 'bundle',
    'asset_unique_id', 'number_sales'
]
